# Imports

# Design

The purpose of this notebook is to outline the different options for possible ways of splitting up the structures

The idea is that each section will describe how to take an input set of c

# Random

# Ligand-based : only the identity of the ligand matters

# Structure-based : only the identity of the structure matters

# Ligand + Structure : Ligand Similarity

# Ligand + Structure : Active Site Similarity